In [1]:
import os

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib.pyplot as plt
import matplotlib.cm as com
import numpy as np

from scipy.signal import savgol_filter, find_peaks
from scipy.spatial import distance

from random import shuffle, uniform
import errno

In [2]:
# Standard Brian2 import
import brian2 as b2

b2.set_device('cpp_standalone')

import matplotlib.pyplot as plt
import os

import time
import pickle


b2.defaultclock.dt = 0.05*b2.ms
clock_dt = b2.defaultclock.dt

%matplotlib inline


In [3]:
include_state_monitor = {}
include_state_monitor['Input'] = False
include_state_monitor['Neurons'] = False
include_state_monitor['Small'] = False
include_state_monitor['Synapses'] = False
include_state_monitor['Output'] = False

In [4]:
in_rate = 100
vth_ = -57

training_foldername = 'training_output'

In [5]:
## INPUT

N_per_edge = 20                         #number of input neurons per dimension
refract_in = 0                          #input neurons refractory period (ms)
in_zone_d = (1/(N_per_edge-1)) + (1e-6) #distance between two consecutive points 
                                        #in a straight line with N_per_edge 
                                        #equidistant points distributed along the line

## WTA

N_o = 61                                #number of neurons per sensor
resting_time = 50                       #resting time (ms)
V_rest = -65                            #resting potential (mV)
V_reset = -65                           #reset potential (mV)
V_th = {}                           
V_th['Gyroscope'] = vth_                #gyroscope neurons base threshold potential time (mV)
V_th['Accelerometer'] = vth_            #accelerometer neurons base threshold potential time (mV)
tau_r = 10                              #refractory period  (ms)
d_vt = 3.0                              #adaptive threshold increment (mV)
tauvt = 400                             #adaptive threshold decay time constant (ms)
tau_m = 30                              #membrane time constant (ms)
tau_I_m = 5                             #input excitatory current time constant (ms)
tau_I_i = 20                            #input inhibitory current time constant (ms)
R_m = 1                                 #excitatory membrane resistance (ohm)
R_i = 1                                 #inhibitory membrane resistance (ohm)


## INPUT -> WTA (excitatory synapses with trace-based STDP)

taupre = 20.0                       #presynaptic activity trace time constant (ms) 
taupost = 20.0                      #postsynaptic activity trace time constant (ms)
Apre = 0.02                         #maximum weight modification for LTP
Apost = - 1.05 * Apre               #maximum weight modification for LTD

w_e = 20.0                          #excitatory current increment  (mA)
w_i = 1                             #inhibitory current increment  (mA)
wmax = 1.0                          #maximum value of weights between input and wta layer
initial_weight_coef = 0.15          #the weights are initialized randomly with values in [0, initial_weight_coef]


## WTA -> WTA (lateral inhibitory synapses)

tau_inh = 10                        #inhibition duration (ms)

In [6]:
V_rest *= b2.mV
V_reset *= b2.mV
V_th['Gyroscope'] *= b2.mV
V_th['Accelerometer'] *= b2.mV
#V_th *= mV
tau_m *= b2.ms
tau_I_m *= b2.ms
tauvt *= b2.ms
tau_I_i *= b2.ms
R_m *= b2.ohm
R_i *= b2.ohm
w_e *= b2.mA
w_i *= b2.mA
tau_r *= b2.ms
d_vt *= b2.mV
taupre *= b2.ms
taupost *= b2.ms
tau_inh *= b2.ms
refract_in *= b2.ms
resting_time *= b2.ms
in_rate *= b2.Hz


In [7]:

if include_state_monitor['Input'] or include_state_monitor['Neurons'] or include_state_monitor['Output']:
    n_samples = 2
    n_reps_per_sample = 1
else:
    n_samples = 50
    n_reps_per_sample = 1

In [8]:
#Evenly distributed neurons within a cubea
class Nube:
    i = 0
    def __init__(self, n_nodes):
        self.n = n_nodes**3
        self.neighbour_dst = 2 / (n_nodes - 1)
        
        ps = np.linspace(-1,1,n_nodes)
        self.coordinates = [[[(x_,y_,z_) for z_ in ps] for y_ in ps] for x_ in ps]
        self.coordinates = [item for sublist in self.coordinates for item in sublist]
        self.coordinates = [item for sublist in self.coordinates for item in sublist]
        self.coordinates = {index: pos for index, pos in enumerate(self.coordinates)}
        self.x = [pos[0] for pos in self.coordinates.values()]
        self.y = [pos[1] for pos in self.coordinates.values()]
        self.z = [pos[2] for pos in self.coordinates.values()]


            
                    
    def scatter_neurons(self, ax):
        for n in range(self.n):
            x_,y_,z_ = self.coordinates[n]
            ax.scatter(x_,y_,z_, alpha = 0.5, c = 'cornflowerblue')
    plt.show()

In [9]:
def load_trial(dir_):
    
    g_x = []
    g_y = []
    g_z = []
    a_x = []
    a_y = []
    a_z = []
    tt = []

    file1 = open(dir_, 'r')

    lines = file1.readlines()

    for line in lines:
            word = line.split()
            try:
                    g_x.append(float(word[0]))
                    g_y.append(float(word[1]))
                    g_z.append(float(word[2]))
                    a_x.append(float(word[3]))
                    a_y.append(float(word[4]))
                    a_z.append(float(word[5]))
                    tt.append(float(word[6]))
            except:
                    pass
    file1.close()

    #data = pd.DataFrame([g_x,g_y,g_z, a_x,a_y,a_z])
    #data = data.transpose()
    #data.columns = ['gx', 'gy', 'gz', 'ax', 'ay', 'az']

    sensor_data = [g_x,g_y,g_z, a_x,a_y,a_z,tt]

    #for a,axis in enumerate(sensor_data):
    #    if a < 6:
    #        sensor_data[a][0] = 0
    #        sensor_data[a] = [value - axis[v-1] for v,value in enumerate(axis)]
            
    return [g_x,g_y,g_z, a_x,a_y,a_z,tt] #, data

def normalize_axis(s_axis):
    
    ret = [[] for s in s_axis]
    
    ret = s_axis.copy()
    
    for a,axis in enumerate(ret):
        for v, value in enumerate(axis):
            
            if ret[a][v] < 0:
                ret[a][v] = ret[a][v]/(32768)    # + 32767)
            else:
                ret[a][v] = ret[a][v]/(32767)   #+ 32768)
    
    return ret
        

In [10]:
dir_ = os.path.join(os.getcwd(), 'Dataset')
n_axis = 6

In [11]:
sample_frequency = 400*b2.Hz
desired_frequency = 100*b2.Hz

downFreqN = int(sample_frequency/desired_frequency)


In [12]:
idxs = [[] for m in range(len(os.listdir(dir_)))]
movements_name = [[] for m in range(len(os.listdir(dir_)))]
samples = [[[] for n in range(n_axis)] for m in range(len(os.listdir(dir_)))]

series = [[] for m in movements_name]
data = [[] for m in movements_name]

for m, movement in enumerate(os.listdir(dir_)):

    #fig = plt.figure(figsize=(10, 9))
    #axs = []
    #for a in range(6):
    #    axs.append(fig.add_subplot(6, 1 , a+1))
    
    
    print(movement)
    movements_name[m] = movement
    
    movement_path = os.path.join(dir_, movement)
    
    n_trials = len(os.listdir(movement_path))
    
    idxs[m] = [[] for n in range(n_trials)]

    if 'Jumping' in movement:
        is_sin_like_axis = 0
        n_filters = 1
    elif 'Torso' in movement:
        is_sin_like_axis = 2
        n_filters = 1
    elif 'Running' in movement:
        is_sin_like_axis = 2
        n_filters = 1
    elif 'Touch' in movement:
        is_sin_like_axis = 2
        n_filters = 1
    else:
        print(movement)

    for tri, trial in enumerate(sorted(os.listdir(movement_path))):


        #[gx, gy, gz, ax, ay, az, t], _ = load_trial(os.path.join(movement_path, trial))
        [gx, gy, gz, ax, ay, az, dt] = load_trial(os.path.join(movement_path, trial))


        
        sensors_axis = [gx, gy, gz, ax, ay, az]
        sensors_axis = normalize_axis(sensors_axis)

        # is_sin_like_axis = index of axis in sensor_axis (0 : g_x, 1 : g_y, ..., 5 : a_z)
        # n_filters : number of times axis is filtered                
        

        #fig = plt.figure(figsize=(30, 20))
        #axs = []

        

        init_th = .3
        init = 0
        for i, val in enumerate(sensors_axis[is_sin_like_axis]):
            if val > init_th:
                init = i
                break
            
        _axis = [[] for a in range(len(sensors_axis))]
        for a,_ in enumerate(sensors_axis):
            _axis[a] = sensors_axis[a][init:]

        #for a, axis in enumerate(_axis):
        #    axs.append(fig.add_subplot(6, 1 , a+1))
        #    axs[-1].plot(np.arange(len(axis)), axis)
        
        filtered = savgol_filter(_axis[is_sin_like_axis], 51, 3)
        
        
        for n in range(n_filters):
            filtered = savgol_filter(filtered, 51, 3)

        #axs[is_sin_like_axis].plot(np.arange(len(filtered)), filtered)
        
        peaks = find_peaks(filtered, height=0)[0][1:-2]
        #diffs = [peaks[p+1]-peak for p, peak in enumerate(peaks[:-1])]
        #for p in peaks:
        #    axs[is_sin_like_axis].axvline(x=p, lw=0.75, c='black')

        #plt.show()


        
        #for p, peak in enumerate(peaks[2:-2]):
        #    idxs[m][t].append((peak, peaks[p+2]))
        
        avg_peak_dst = sum([peaks[p+1]-peak for p,peak in enumerate(peaks[:-1])])/(len(peaks)-1)
        print(avg_peak_dst)

        for p, peak in enumerate(peaks[:-1]):
            series_i= np.zeros((len(_axis), int(peaks[p+1]-peak)))
            for a, axis in enumerate(_axis):
                if peaks[p+1]-peak>0.85*avg_peak_dst and peaks[p+1]-peak<1.15*avg_peak_dst: 
                    series_i[a] = np.array(axis[peak:peaks[p+1]])

                    sample = np.array(axis[peak:peaks[p+1]])
                    #diff_sample = [value - sample[v-1] for v,value in enumerate(sample[1:], start=1)]
                    #sample = [0 for val in sample]
                    samples[m][a].append(sample)

                    #axs[a].plot(np.arange(len(sample)), sample)
            series[m].append(series_i)
            
    #plt.show()
        
        

Running in place
86.57894736842105
83.97560975609755
86.23076923076923
85.05263157894737
85.55263157894737
81.7948717948718
Torso rotation
189.48888888888888
179.02564102564102
179.57894736842104
174.16666666666666
179.28205128205127
171.5
Jumping jacks
91.70454545454545
93.62162162162163
94.10810810810811
89.07894736842105
92.25
91.32432432432432
Touch feet
174.3846153846154
159.6153846153846
166.78571428571428
162.8
156.35714285714286
160.64285714285714
159.42857142857142
161.42857142857142
167.57142857142858
166.76923076923077
165.76923076923077
173.07142857142858
165.5
166.30769230769232
167.92857142857142
169.23076923076923


In [13]:
for m,movement in enumerate(movements_name):
    print(f'Movement {movement} has {len(samples[m][0])} samples')

Movement Running in place has 229 samples
Movement Torso rotation has 254 samples
Movement Jumping jacks has 224 samples
Movement Touch feet has 219 samples


In [14]:
sensors = {'Gyroscope':[0,1,2], 'Accelerometer':[3,4,5]}
axis = {0:'x', 3:'x',1:'y', 4:'y',2:'z', 5:'z'}

In [15]:

resting_array = [0.0 for _ in range(int(resting_time/(1/desired_frequency)))]


nu = Nube(N_per_edge)

N_i = nu.n
Input = {}
InputSpikeMonitor = {}
InputStateMonitor = {}
InputMonitor = {}
InputRateMonitor = {}

Synapse = {}



pattern_dt = 1/desired_frequency
elapsed_time=0*b2.ms
x = {sensor : [] for sensor in sensors}
y = {sensor : [] for sensor in sensors}
z = {sensor : [] for sensor in sensors}


begins = []
durations = []
movement_index = []

iteration = []
it = 1

iteration_begin = []

movement_it = []

learning = []

events = [0,1,2,3,4,5]
event_type = {sensor: [] for sensor in sensors}



#for n in range(n_reps_per_sample):
for sp in range(n_samples):
    for m,movement in enumerate(movements_name):
        for s,sensor in enumerate(sensors):
            
            axis_x = sensors[sensor][0]
            axis_y = sensors[sensor][1]
            axis_z = sensors[sensor][2]

            x[sensor].extend(samples[m][axis_x][-1-sp])
            x[sensor].extend([x[sensor][-1] for r in range(len(resting_array))])

            y[sensor].extend(samples[m][axis_y][-1-sp])
            y[sensor].extend([y[sensor][-1] for r in range(len(resting_array))])

            z[sensor].extend(samples[m][axis_z][-1-sp])
            z[sensor].extend([z[sensor][-1] for r in range(len(resting_array))])




        sample_duration = len(samples[m][axis_x][-1-sp])
        
        learning.extend([1.0 for _ in range(int(sample_duration))])
        learning.extend([-1 for _ in range(len(resting_array))])

        iteration.extend([it for _ in range(int(sample_duration))])
        iteration.extend(resting_array)

        iteration_begin.extend([elapsed_time for _ in range(int(sample_duration))])
        iteration_begin.extend(resting_array)

        movement_it.extend([m for _ in range(int(sample_duration))])
        movement_it.extend([m for _ in range(len(resting_array))])

        begins.append(elapsed_time)
        durations.append(sample_duration*pattern_dt)
        movement_index.append(m)

        elapsed_time += sample_duration * pattern_dt + resting_time

    

    it += 1

is_learning = b2.TimedArray(learning, dt=pattern_dt)




In [16]:
x_sensor_0 = b2.TimedArray(x['Gyroscope'], dt=(1/desired_frequency))
y_sensor_0 = b2.TimedArray(y['Gyroscope'], dt=(1/desired_frequency))
z_sensor_0 = b2.TimedArray(z['Gyroscope'], dt=(1/desired_frequency))

x_sensor_1 = b2.TimedArray(x['Accelerometer'], dt=(1/desired_frequency))
y_sensor_1 = b2.TimedArray(y['Accelerometer'], dt=(1/desired_frequency))
z_sensor_1 = b2.TimedArray(z['Accelerometer'], dt=(1/desired_frequency))


In [17]:
movement_duration = [0 for m in movements_name]
for m,movement in enumerate(movements_name):        

    movement_duration[m] = len(samples[m][0])*pattern_dt



In [18]:
#training_dir = 'training_output'
training_dir = training_foldername
weights_dt = 5*b2.second

In [19]:
import scipy

In [20]:
def get_accuracy(time, voltage, nr_connections):

    colors = ['red', 'green', 'blue', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']


    starts = [begin for begin in begins if begin < time[-1]]

    max_v_per_iteration = {n:{s: -1  for s in range(len(starts))} for n in range(len(movements_name))}

    ths = {}

    accuracy = {m: 0 for m in range(len(movements_name))}
    false_pos = {m: {n: 0 for n in range(len(movements_name))} for m in range(len(movements_name))}

    for s,start_time in enumerate(starts):
        
        end_time = start_time + durations[s]
        start_idx = time.searchsorted(start_time)
        end_idx = time.searchsorted(end_time)


        maxs = {}
        for neuron, v in enumerate(voltage):
            maxs[neuron] = max(v[start_idx:end_idx])/nr_connections[neuron]
        
        max_v = max(maxs.values())


        if max_v > 0.0:
            max_neuron = list(maxs.values()).index(max_v)
            if max_neuron == movement_index[s]:
                accuracy[movement_index[s]] += 1
            else:
                false_pos[movement_index[s]][max_neuron] += 1

    print(accuracy)
    print(false_pos)

    for m in range(len(movements_name)):
        accuracy[m] /= n_samples
        accuracy[m] *= 100
        for n in range(len(movements_name)):
            false_pos[m][n] /= n_samples
            false_pos[m][n] *= 100


    print(accuracy)
    print(false_pos)




    
    return accuracy, false_pos


In [21]:
weights= {}

In [22]:
#   This function implements the two possible testing iterations
#   delays : synaptic delays calculated in first iteration
#   spikes : wta neurons spikes recorded from last iteration
#
#   If delays and spikes are empty, the function will perform the first iteration. Otherwise it performs the second. The testing set is used for both iterations.
#
#   First iteration     :   determining wta --> recognition connections and corresponding delays.
#                           a network similar to the learning stage is used. 
#                           for each pattern presentation, the first spike from each wta neuron is considered.
#                           for each pattern, if a wta neuron shows a similar firing time (relative to the start of a pattern)
#                           across all the testing set, a delayed connection is established between this wta neuron and the corresponding 
#                           recognition neuron. The requirement for a wta to be selected are:
#                               - it must fire 90% of the times for a specific movement
#                               - its firing times must all be close enough to the mean of all the firing times (for a specific movement)
#                           the delays are calculated from the mean of firing times. the thesis document details how
#                           
#                           
#   Second iteration    :   assess the network performance.
#                           a simpler network is used with just two layers: wta and recognition.
#                           the wta layer is implement with a SpikeGeneratorGroup. the wta firing times were recorded it the first iteration.
#                           there are as many recognitions neurons as the number of movements (4 in this case).
#                           each movement pattern presentantion is said to be correctly recognized if its corresponding recognition neuron 
#                           exhibits the highest membrane potential during the pattern presentation 
                          

def simulate(delays={}, spikes={}):

    

    b2.device.reinit()
    b2.device.activate()

    testing_delays = False                  
    testing_accuracy = False

    if delays=={} and spikes=={}:
        # determining wta --> recognition connections and corresponding delays
        
        # not sure if cpp_standalone accelerates the simulation
        # can use runtime as well
        b2.set_device('cpp_standalone')
        testing_delays = True
    if delays!={} and spikes!={}:
        # assess the network performance
        b2.set_device('runtime')        
        testing_accuracy = True
    
    
    net = b2.Network()

    # wta neuron equations
    neuron_eqs = '''
    dv/dt = ((V_rest-v) + R_m*I_m - R_i*I_i)/tau_m : volt (unless refractory)
    dI_m/dt = -I_m/tau_I_m : amp
    dI_i/dt = -I_i/tau_I_i : amp
    dvt/dt = (Vth-vt)/(tauvt) : volt
    inhibited : boolean
    Vth : volt
    '''
    reset = '''
    vt = v + d_vt
    v=V_reset
    I_m = 0*amp
    '''

    neurons = {}
    NeuronsSpikeMonitor = {}
    NeuronsStateMonitor = {}
    inhibition = {}
    inhibition_0 = {}

    

    InputNeuronsSynapse = {}
    InputNeuronsSynapseMonitor = {}

    if testing_delays:
        # set up a network similar to training
        # refer to the training notebook for comments on this
        for s, sensor in enumerate(sensors):
            Input[sensor] = b2.NeuronGroup(N_i, 
                                            '''
                                            zone_fire : Hz
                                            min_fire = 0.05*Hz : Hz
                                            x : 1   (constant)
                                            y : 1   (constant)
                                            z : 1   (constant)
                                            x_t = x_sensor_''' +str(s)+ '''(t) : 1 (shared)
                                            y_t = y_sensor_''' +str(s)+ '''(t) : 1 (shared)
                                            z_t = z_sensor_''' +str(s)+ '''(t) : 1 (shared)
                                            er = (abs(x_t-x)/2) + (abs(y_t-y)/2) + (abs(z_t-z)/2) : 1
                                            in_zone = er <= in_zone_d*sqrt(3) : boolean   (constant over dt)
                                            ''',
                                            threshold='(rand()<min_fire*dt or (in_zone and rand()<zone_fire*dt)) and is_learning(t)==1',
                                            refractory=refract_in,
                                            method='euler')

            Input[sensor].zone_fire = in_rate
            Input[sensor].x = nu.x
            Input[sensor].y = nu.y
            Input[sensor].z = nu.z
            
            net.add(Input[sensor])
            
            InputSpikeMonitor[sensor] = b2.SpikeMonitor(Input[sensor])
            net.add(InputSpikeMonitor[sensor])

            neurons[sensor] = b2.NeuronGroup(N_o, model=neuron_eqs,
                                threshold='v>vt and not(inhibited)', reset=reset,
                                refractory='tau_r', method='euler')

            neurons[sensor].v = 'V_rest'
            neurons[sensor].vt = V_th[sensor]
            neurons[sensor].Vth = V_th[sensor]

            neurons[sensor].I_m = '0*amp'
            neurons[sensor].I_i = '0*amp'
            neurons[sensor].inhibited = False
            neurons[sensor].run_regularly('v = v*int(is_learning(t)==1) + V_rest*int(not(is_learning(t)==1))', dt=pattern_dt)
            neurons[sensor].run_regularly('vt = vt*int(is_learning(t)==1) + Vth*int(not(is_learning(t)==1))', dt=pattern_dt)
            


            net.add(neurons[sensor])

            NeuronsSpikeMonitor[sensor] = b2.SpikeMonitor(neurons[sensor])
            net.add(NeuronsSpikeMonitor[sensor])

            inhibition[sensor] = b2.Synapses(neurons[sensor], neurons[sensor], 
                                        on_pre='''I_i_post = clip(I_i_post + w_i, 0*amp, I_i_post + w_i)
                                                inhibited_post = True''')
            inhibition_0[sensor] = b2.Synapses(neurons[sensor], neurons[sensor], 
                                            on_pre='inhibited_post = False')

            inhibition[sensor].connect('i!=j')
            inhibition_0[sensor].connect('i!=j')
            inhibition_0[sensor].delay = tau_inh

            net.add(inhibition[sensor])
            net.add(inhibition_0[sensor])

            

            # the weights are now static and equal to a set if recorded weights during training
            InputNeuronsSynapse[sensor] = b2.Synapses(Input[sensor], neurons[sensor],  'w : 1',
                                                on_pre='I_m_post = I_m_post + w*w_e*(not_refractory_post)*int(not(inhibited_post))')

            InputNeuronsSynapse[sensor].connect()
            InputNeuronsSynapse[sensor].w = weights[sensor]     # assign the recorded weights (explained below)
            net.add(InputNeuronsSynapse[sensor])   

    if testing_delays:

        # to estimate the delay we'll just return the wta firings
        # we also return the input spikes if there's anything we want to do with it

        net.run(elapsed_time)
        
        delays_ = {sensor : {output_neuron :{ input_neuron: -1 for input_neuron in range(N_o)} for output_neuron in range(len(movements_name))} for sensor in sensors}        
        
        return delays_, {sensor: {'i':InputSpikeMonitor[sensor].i[:],'t':InputSpikeMonitor[sensor].t[:]} for sensor in sensors},  {sensor: {'i':NeuronsSpikeMonitor[sensor].i[:],'t':NeuronsSpikeMonitor[sensor].t[:], 'spike_trains':NeuronsSpikeMonitor[sensor].spike_trains()} for sensor in sensors}, net

    elif testing_accuracy:

        # set up recognition layer
        # leaky and integrate neurons

        
        NeuronsOutputSynapse= {}
        output_tau = [40, 40, 40, 40]*b2.ms
        output_vth = [100, 100, 100, 100]       # v never > 100 --> neurons don't fire. can also remove threshold condition
        


        output = b2.NeuronGroup(len(movements_name), '''dv/dt = -v/tau : 1
                                                        vth : 1
                                                        tau : second''', threshold='v > vth', reset='v=0', method='exact')
        output.vth = output_vth
        output.tau = output_tau
        OutputSpikeMonitor = b2.SpikeMonitor(output)
        net.add(output)
        net.add(OutputSpikeMonitor)
        #OutputStateMonitor = b2.StateMonitor(output, ['v'] ,record=True)
        #net.add(OutputStateMonitor)

        connected = False #indicates if any of the sensors has connections

        n_connections = {m: 0 for m in range(len(movements_name))}
        for sensor in sensors:

            # create a SpikeGeneratorGroup with the wta firing times of the first testing interation
            neurons[sensor] = b2.SpikeGeneratorGroup(N_o, spikes[sensor]['i'], spikes[sensor]['t'])            
            net.add(neurons[sensor])           
            

            # create synapses between wta and recognition neurons
            NeuronsOutputSynapse[sensor] = b2.Synapses(neurons[sensor], output, 
                                                     '''w : 1''',
                                                        on_pre = ''' v_post += w
                                                        ''')
            
            sconnected = False  #indicates if this sensor has connections
            
            # for each wta neuron
            for n in range(N_o):
                # for each recognition neuron                                                            
                for m in range(len(movements_name)): 
                    # each delay in 'delays' is initiallized without units
                    # if it has units of type second, it means there is a delayed connection 
                    # between wta neuron with index n and recognition neuron with index m
                    if type(delays[sensor][m][n]) == type(0*b2.ms):
                        
                        NeuronsOutputSynapse[sensor].connect(i=n, j=m)
                        NeuronsOutputSynapse[sensor].delay[n,m] = delays[sensor][m][n]

                        n_connections[m] += 1   # we save the number of connections per sensor
                                                # to later divide each recognition neuron potential
                                                # by the number of connections it receives

                        sconnected = True       # the are connections for this sensor
                        
            # if the sensor has connections
            if sconnected:
                NeuronsOutputSynapse[sensor].w = 1   

                net.add(NeuronsOutputSynapse[sensor])
            
            connected = connected or sconnected


        # record the potential of recognitions neurons 
        OutputStateMonitor = b2.StateMonitor(output, ['v'], record=np.arange(output.N), dt= 0.5*b2.ms)
        net.add(OutputStateMonitor)

        net.run(elapsed_time)

        # get_accuracy() will check if each recognition neuron had the highest potential
        # when presented with its corresponding movement
        # acc : % times each recognition neuron fired to its corresponding movement
        # flspstv : false positives (a recognition neuron firing for another movement)
        acc, flspstv = get_accuracy(OutputStateMonitor.t[:], OutputStateMonitor.v[:], n_connections)
        del net

        return (acc, flspstv)

In [23]:
# 'time_window_begin' : list stores the start time of each pattern presentation
# 'dts'               : list stores the duration time of each pattern
# 
#  Return the index i of the element in 'time_window_begin' such that time_instant is in the interval [time_window_begin[i], time_window_begin[i]+dts[i]]
def get_b(time_instant, time_window_begin, dts):
    for ti in range(len(time_window_begin)):
        if time_instant >= time_window_begin[ti] and time_instant <= time_window_begin[ti] + dts[ti]:
            return ti    
    
    return -1

In [24]:
clrs = ['red', 'green', 'black', 'orange']

In [25]:
def test():
    

    d = []
    results = []

    # we'll load the weights from the training output
    # first we see how many weight recordings there are
    # the file has following structure:
    # recording_time0 weight0 weight1 weight 2 ...
    # recording_time1 weight0 weight1 weight 2 ...
    #
    # n_ws : number of recordings
    # last_w_dt : 
    with open(os.path.join(training_dir, 'weights_'+'Gyroscope'+'.txt'), 'r') as f:
        for l,line in enumerate(f):
            pass
            
        n_ws = l + 1
        last_w_dt = int(line.split()[0])

    

    
    # the recordings time instants
    w_recordings = np.linspace(0, last_w_dt, n_ws,endpoint=True, dtype=int)

    # the recordings occurs at every 5 seconds (can be changed in the training notebook)
    # which yields a large number of recordings
    # to test a smaller number of recordings, or a specific recording
    # we should determine the recording time instants we want to test
    # and save it in w_recordings. note that the available time instants are multiples of the 
    # weight recording time period (in this case 0, 5, 10, 15... seconds)
    # just manipulate the existing values in w_recordings to include the recordings you want
    
    # w_recordings = [w_recordings[0]]   -> tests the first weight recording
    # w_recordings = [w_recordings[-1]]  -> tests the last weight recording
    # w_recordings = w_recordings[::20]  -> tests every 20th the weight recording 

    # another way is to set a new weight recording time period
    # and get the corresponding slicer

    # recording_dt = 5*b2.second
    # test_w_dt = 40*b2.second
    # new_recording_dt_k = int(test_w_dt/recording_dt)
    # for r, recording in enumerate(w_recordings[::new_recording_dt_k]):
    #   ...
    # 
    # here we're just going to test against the last recording

    w_recordings = [w_recordings[-1]]
    
    # perform test for each considered weight recording
    for r, recording in enumerate(w_recordings):
    
        print(f'Recorded at {recording} s (/{last_w_dt} s)')

        
        
        delays_ = {sensor : {output_neuron :{ input_neuron: -1 for input_neuron in range(N_o)} for output_neuron in range(len(movements_name))} for sensor in sensors}
        delays = {sensor : {output_neuron :{ input_neuron: -1 for input_neuron in range(N_o)} for output_neuron in range(len(movements_name))} for sensor in sensors}

        # for each sensor, fetch the weight recording at save it in 'weights'
        for sensor in sensors:
            with open(os.path.join(training_dir, 'weights_'+sensor+'.txt'), 'r') as f:
                for l,line in enumerate(f):
                    if int(line.split()[0])==recording:
                        weights[sensor] = [float(w_) for w_ in line.split()[1:]]
                        break
                    else:
                        pass

        # perform first testing iteration
        # get delayed connections and wta firing times           
        delays, inputSpikes, wta_spikes, net_ = simulate()
        del net_        

        
        connected = False       # will be True if there are connections for any of the sensors
        for sensor in sensors:

            spiketrains = wta_spikes[sensor]['spike_trains']    #wta neuron spike times            
            first_spikes_by_neuron = {m: {n: {} for n in range(N_o)} for m in range(len(movements_name))}

            # for each pattern presentation
            for b, start_time in enumerate(begins):

                end_time = start_time + durations[b]                
                m = movement_index[b]

                # for each wta neuron, get the first spike that lies in the interval [start_time, end_time]
                for j in range(len(spiketrains)):
                    neuron_spikes = spiketrains[j]
                    spikes_in_interval = neuron_spikes[(neuron_spikes >= start_time) & (neuron_spikes < end_time)]

                    # if there is any spike, save the firing time difference 
                    # since the beggining of the pattern presentation
                    if len(spikes_in_interval) > 0:
                        first_spike_time = spikes_in_interval[0]
                        first_spikes_by_neuron[m][j][b] = first_spike_time - start_time

            # now we have the the time it took for each wta neuron to fire after being presented with a specific movement
            # the first spikes are saved as per movement/ per wta neuron index / per index of pattern presentation
            # we'll organize them per wta neuron index / per index of movement presented
            sconnected = False # will be True if there are connections for the current sensor
            for j in range(N_o):
                
                # get wta neuron number number of fires for each movement (each movement is presented n_samples times, i.e. has n_samples patterns)
                n_fires = [len(list(first_spikes_by_neuron[m][j].values())) for m in range(len(movements_name))]
                # check if wta neuron j fired 90% of times for each movement
                n_fires_own = [n_fires[m]>0.9*n_samples for m in range(len(movements_name))]
                
                # for each movement
                for mov_index, f in enumerate(n_fires_own):
                    # if neurons fired 90% times
                    if f==True:
                        # calculate the average firing time of neuron j for movement with index mov_index
                        spikes = list(first_spikes_by_neuron[mov_index][j].values()) 
                        average_firing_time = sum(spikes)/len(spikes)

                        # calculate the deviation of firing times relative to the average firing time
                        dts = [abs(spike-average_firing_time) for spike in spikes]                        
                        dtmean = (sum(dts)/len(dts))/pattern_dt

                        # if the deviation is small enough
                        if dtmean < 15:
                            # set the delay to equal to the average firing time
                            delays_[sensor][mov_index][j] = average_firing_time

        
        # we need to calculate the final delays
        # for each movement, we get the highest average firing time (consider it is the neuron with index n)
        # and add a delay to the remaining neurons such that :
        #   highest_average_firing_time = wta_average_firing_time[n]
        #   wta_average_firing_time[j] + delay[j] = highest_average_firing_time <=> delay[j] = highest_average_firing_time - wta_average_firing_time[j]
        mconnected = {}
        for  m in range(len(movements_name)):
            movement_delays = [[d for d in delays_[sensor][m].values() if type(d)==type(0*b2.ms)] for sensor in sensors]

            movement_delays = [delay for sensor in movement_delays for delay in sensor]

            if len(movement_delays) > 0:
                max_fire = max(movement_delays)
                for n in range(N_o):
                    for sensor in sensors:
                        d = delays_[sensor][m][n]
                        if type(d)==type(0*b2.ms):
                            delays[sensor][m][n] = max_fire - d
                            print(sensor, m,n, delays[sensor][m][n])
                            
                            mconnected[m] = True
            else:
                mconnected[m] = False
                        
                
        mconnected = list(mconnected.values())
        connected = all(mconnected)
        
        # if the simulation yielded results i.e. any wta neuron showed a regular behavior
        #                                        and delays were determined 
        if connected:
            # perform second test iteration
            # get accuracy and save it
            result = simulate(delays, wta_spikes)
            results.append(result)

        # if no delayed connections were established
        # save an empty result
        else:
            for m, conn in enumerate(mconnected):
                if not(conn):
                    print(f'Movement {movements_name[m]} has no connections')
            
            accu = {m: 0 for m in range(len(movements_name))}
            falsep = {m: {n: 0 for n in range(len(movements_name))} for m in range(len(movements_name))}

            results.append((accu, falsep))
        

    return results

In [26]:
r = test()

Recorded at 815 s (/815 s)


WARNING    "i" is an internal variable of group "synapses", but also exists in the run namespace with the value 271. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    "i" is an internal variable of group "synapses_1", but also exists in the run namespace with the value 271. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    "i" is an internal variable of group "synapses_3", but also exists in the run namespace with the value 271. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    "i" is an internal variable of group "synapses_4", but also exists in the run namespace with the value 271. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]


Accelerometer 0 0 0.3793551 s
Accelerometer 0 5 0.5003231 s
Gyroscope 0 6 0.4432911 s
Accelerometer 0 8 0.3307131 s
Accelerometer 0 12 0.4818151 s
Accelerometer 0 14 0.41670204 s
Accelerometer 0 16 0.35378571 s
Gyroscope 0 17 262.93710204 ms
Accelerometer 0 19 0.3618851 s
Gyroscope 0 23 0.43742802 s
Accelerometer 0 23 0.4717271 s
Gyroscope 0 24 167.29423248 ms
Gyroscope 0 26 0. s
Accelerometer 0 27 0.44044677 s
Accelerometer 0 29 0.3805511 s
Accelerometer 0 30 291.79910204 ms
Accelerometer 0 31 83.41510204 ms
Gyroscope 0 34 0.32539978 s
Accelerometer 0 37 169.61632653 ms
Gyroscope 0 38 0.44315714 s
Accelerometer 0 38 0.43202802 s
Accelerometer 0 39 3.69339991 ms
Accelerometer 0 42 0.4374731 s
Gyroscope 0 43 202.79310204 ms
Gyroscope 0 44 145.78218537 ms
Accelerometer 0 45 0.43117249 s
Accelerometer 0 46 0.3599871 s
Accelerometer 0 48 0.4172371 s
Accelerometer 0 49 288.98510204 ms
Accelerometer 0 53 0.3389951 s
Gyroscope 0 58 0.43902684 s
Accelerometer 0 58 0.46085302 s
Gyroscope 1 0 0.

In [27]:
r

[({0: 100.0, 1: 100.0, 2: 100.0, 3: 98.0},
  {0: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0},
   1: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0},
   2: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0},
   3: {0: 2.0, 1: 0.0, 2: 0.0, 3: 0.0}})]

In [28]:
# This function will plot the evolution of the network recognition performance overtime
def plot_results(results):

    clrs = ['red', 'green', 'orange']
    ls = ['solid','dashed', 'dashdotdotted']
    markers = ["o", "D", "s", '']

    fp = {}

    for neuron in range(len(movements_name)):
        acc_neuron = [(r_,result[0][neuron]) for r_,result in enumerate(results)]
        x, y = zip(*acc_neuron)
        

        plt.scatter(x,y, c='black')

        fp_neuron = [result[1] for result in results]

        fp[neuron] = {m: [] for m in range(len(movements_name)) if m!=neuron}

        for f, fp_ in enumerate(fp_neuron):
            for m in range(len(movements_name)):
                if m != neuron:
                    fp[neuron][m].append(fp_[m][neuron])
                    plt.plot(f, fp_[m][neuron], c=clrs[neuron], ms=5, alpha=.4)
                    #plt.scatter(f, fp_[m][neuron], marker=markers[m],c=clrs[neuron])
    

    
    acc = [sum(list(acc_.values()))/len(movements_name) for (acc_, _) in results]
    plt.plot(np.arange(len(acc)), acc, 'o-b', ms=5)
    plt.show()


In [29]:
if len(r) > 1:
    plot_results(r)
else:
    print(r)

[({0: 100.0, 1: 100.0, 2: 100.0, 3: 98.0}, {0: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}, 1: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}, 2: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}, 3: {0: 2.0, 1: 0.0, 2: 0.0, 3: 0.0}})]


In [30]:
saveResults = False

In [31]:
if saveResults:
    with open(os.path.join(training_dir,"results.pickle"), "wb") as file:
        pickle.dump(r, file)